In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [10]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [12]:
# Ecode Categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [13]:
# One hot encode 'Geography'

one_hot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
# Combine one-hot encoded columns with original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
# Split the data into features and target

X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# Save the encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

    
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)

In [18]:
# ANN Regression Problem Statement

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [20]:
# Build the model
model = Sequential(
     [
          Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)),
          Dense(32, activation = 'relu'),
          Dense(1) # Output layer for regression
     ]
)

# Compile the mode

model.compile(optimizer = 'adam', loss='mean_absolute_error', metrics = ['mae'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                832       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [25]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5,
                                        restore_best_weights = True
                                        )


In [26]:
# Train the model
history = model.fit(
     X_train, y_train,
     validation_data = (X_test, y_test),
     epochs = 100,
     callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 8ms/step - loss: 100367.9297 - mae: 100367.9297 - val_loss: 98480.7344 - val_mae: 98480.7344
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99494.3984 - mae: 99494.3984 - val_loss: 96708.6250 - val_mae: 96708.6250
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 96384.2656 - mae: 96384.2656 - val_loss: 92128.5312 - val_mae: 92128.5312
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 90274.1875 - mae: 90274.1875 - val_loss: 84612.2422 - val_mae: 84612.2422
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 81585.7578 - mae: 81585.7578 - val_loss: 75267.5781 - val_mae: 75267.5781
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 71815.3828 - mae: 71815.3828 - val_loss: 66057.5859 - val_mae: 66057.5859
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 62901.5195 - mae: 62901.519

In [37]:
# Evaluate the model
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [40]:
%tensorboard --logdir regressionlogs/fit20250720-104616

Reusing TensorBoard on port 6007 (pid 15396), started 0:00:05 ago. (Use '!kill 15396' to kill it.)

In [41]:
# Evaluate model on the test data

test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE : {test_mae}")

63/63 [==============================] - 1s 5ms/step - loss: 50243.3867 - mae: 50243.3867
Test MAE : 50243.38671875


In [42]:
model.save('regression_model.h5')

c:\Users\mk744\OneDrive - Poornima University\Desktop\ChurnPrediction\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
